In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Ouverture du dataframe TITANIC
df = pd.read_csv('../input/titanic/train_and_test2.csv')
pd.set_option('display.max_columns', None)
df.head()


In [ ]:
#Rapide analyse des différentes varialbes et de la target (2urvived ici, ) : remarque tout de suite que toutes les variables zero sont inutiles et devront être retiré
df.describe()

In [ ]:
#Analyses des différentes variables, tous des float ou des int ==> variables déjà encodé et sont propice à l'utilitsation d'un algorythme de machine learning
df.dtypes.value_counts().plot.pie()

In [ ]:
# Analyse des valeurs manquantes : très peu de valeur manquante, un dropna pour retirer les valeurs manquantes est une bonne idéee vu qu'il y en a peu
plt.figure(figsize=(20,10))
sns.heatmap(df.isna())

In [ ]:
#supprime les variables inutiles
df = df.drop(df.columns[5:12], axis = 1)
df = df.drop(df.columns[6:13], axis = 1)
df = df.drop(['zero.14','zero.15','zero.16','zero.17','zero.18','Passengerid'], axis = 1)
df

In [ ]:
#visualisation de la target : 75% morts et 25% survivant environ ==> problème de manque de donnée pour le train set. En effet celui possède 50% survivant et 50% morts. 
plt.hist(df['2urvived'])

In [ ]:
#Création du dataframe train et visualisation (df_viz)
df_alive = df[df['2urvived']==1].copy()
df_dead = df[df['2urvived']==0][:len(df_alive)].copy()
df_viz = pd.concat([df_alive,df_dead])

pd.set_option('display.max_rows', None, 'display.max_columns', None)
df_viz

In [ ]:
#étude des répartitions de variables selon la target dans le dataframe train. Le sex, la classe, le coût semble avoir un bon impact avec un peu l'age en plus
for col in df.columns:
    plt.figure()
    plt.hist([df_alive[col],df_dead[col]], label=['alive','dead'])
    plt.title(col)
    plt.legend()

In [ ]:
#même analyse mais cette fois ci dans le dataframe entier. On arrive aux même conclusion ==> Dataframe train est une bonne représentation de la réalité
for coll in df.columns:
    g = sns.FacetGrid(df,col='2urvived')
    g.map(plt.hist, coll, bins=50)

In [ ]:
# matrice de correlation  montre de bonne correlation entre la target avec le sex (40%) la classe (-24%) le cou^t (17%) toutefois les correlations sont mauvaises avec le reste.
# Une idée est donc de tenter d'ameliorer la correlation pour ameliorer le modèle
f, ax2 = plt.subplots(1, 1, figsize=(20,10))
sub_sample_corr = df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2,annot = True)
ax2.set_title('matrice correlation', fontsize=14)
plt.show()

In [ ]:
# Definition de la fonction évaluation avec une matrice de confusion, les scores ainsi que la courbe d'apprentissage.
df_alive = df[df['2urvived']==1].copy()
df_dead = df[df['2urvived']==0][:len(df_alive)].copy()
df_viz = pd.concat([df_alive,df_dead])
pd.set_option('display.max_rows', None, 'display.max_columns', None)


df_viz=df_viz.dropna()
df_viz=df_viz.reset_index().drop(['index'],1)
y= df_viz['2urvived']
X = df_viz.drop(['2urvived'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)
clf = KNeighborsClassifier(n_neighbors=5, weights='distance')

def evaluation(model):
    model.fit(X_train,y_train)
    ypred=model.predict(X_test)
    print(confusion_matrix(y_test,ypred))
    print(classification_report(y_test,ypred))
    N, train_score, val_score = learning_curve(model,X_train, y_train, cv=4,scoring= 'f1',train_sizes=np.linspace(0.1,1,10))
    plt.figure(figsize=(10,5))
    plt.plot(N, train_score.mean(axis=1), label='train score')
    plt.plot(N, val_score.mean(axis=1), label='train score')

In [ ]:
# Premier model test simple, résultat satisfaisant cepandant il y a un gros gap entre nos deux courbes dans le graphique des courbes d'apprentissage
model = DecisionTreeClassifier(random_state=0)
evaluation(model)

In [ ]:
#Répartition du poids des variables par de l'algorythme utilisé
pd.DataFrame(model.feature_importances_, index= X_train.columns).plot.bar()

In [ ]:
# Etape Pre-processing : Objectif : catégorisé l'âge pour regrouper plus de donnée
for i in range(len(df_viz)): 
    if df_viz['Age'][i]<= 5:
        df_viz['Age'][i]= 0
    if df_viz['Age'][i]> 5 and df_viz['Age'][i]<= 10:
        df_viz['Age'][i]= 1
    if df_viz['Age'][i]> 10 and df_viz['Age'][i]<= 15:
        df_viz['Age'][i]= 2
    if df_viz['Age'][i]> 15 and df_viz['Age'][i]<= 20:
        df_viz['Age'][i]= 3
    if df_viz['Age'][i]> 20 and df_viz['Age'][i]<= 25:
        df_viz['Age'][i]= 4
    if df_viz['Age'][i]> 25 and df_viz['Age'][i]<= 27:
        df_viz['Age'][i]= 5
    if df_viz['Age'][i]> 27 and df_viz['Age'][i]<= 28:
        df_viz['Age'][i]= 6
    if df_viz['Age'][i]> 28 and df_viz['Age'][i]<= 35:
        df_viz['Age'][i]= 7
    if df_viz['Age'][i]> 35 and df_viz['Age'][i]<= 45:
        df_viz['Age'][i]= 8
    if df_viz['Age'][i]> 45 and df_viz['Age'][i]<= 55:
        df_viz['Age'][i]= 9
    if df_viz['Age'][i]> 55 and df_viz['Age'][i]<= 65:
        df_viz['Age'][i]= 10
    if df_viz['Age'][i]> 65 :
        df_viz['Age'][i]= 11
        


In [ ]:
# test avec l'âge modifié, résultat : meilleur résultat, on garde cette opération
y= df_viz['2urvived']
X = df_viz.drop(['2urvived'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)
clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
evaluation(model)

In [ ]:
## on fait de même mais pour le coût :
#for i in range(len(df_viz)): 
 #   if df_viz['Fare'][i]<= 10:
 #       df_viz['Fare'][i]= 0
 #   if df_viz['Fare'][i]> 10 and df_viz['Fare'][i]<= 20:
#        df_viz['Fare'][i]= 1
#    if df_viz['Fare'][i]> 20 and df_viz['Fare'][i]<= 30:
#        df_viz['Fare'][i]= 2
#    if df_viz['Fare'][i]> 30 and df_viz['Fare'][i]<= 50:
#        df_viz['Fare'][i]= 3
#    if df_viz['Fare'][i]> 50 and df_viz['Fare'][i]<= 70:
#        df_viz['Fare'][i]= 4
#    if df_viz['Fare'][i]> 70 and df_viz['Fare'][i]<= 100:
#        df_viz['Fare'][i]= 5
#    if df_viz['Fare'][i]> 100 and df_viz['Fare'][i]<= 150:
#        df_viz['Fare'][i]= 6
#    if df_viz['Fare'][i]> 150 and df_viz['Fare'][i]<= 200:
#        df_viz['Fare'][i]= 7
#    if df_viz['Fare'][i]> 200 and df_viz['Fare'][i]<= 250:
#        df_viz['Fare'][i]= 8
#    if df_viz['Fare'][i]> 250 and df_viz['Fare'][i]<= 350:
#        df_viz['Fare'][i]= 9
#    if df_viz['Fare'][i]> 350 and df_viz['Fare'][i]<= 450:
#        df_viz['Fare'][i]= 10
#    if df_viz['Fare'][i]> 450 :
# #       df_viz['Fare'][i]= 11

In [ ]:
#résultat : catégoriser le coût nous fait obtenir de moins bon résultat. On supprime cette idée
#y= df_viz['2urvived']
##X = df_viz.drop(['2urvived'],axis=1)
###X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
####model = make_pipeline(SelectKBest(f_classif, k=6), DecisionTreeClassifier(random_state=0))
#####evaluation(model)

In [ ]:
# Test plein d'algorythme différent pour choisir le meilleur
#Résultat : RandomForest possède les meilleurs résultat mais AdaBoostClassifier possède la meilleur courbe d'apprentissage (gap très léger).
# Les deux modèles seront donc étudier et nous allons en retenir le meilleur.
preprocessor = make_pipeline(SelectKBest(f_classif, k=6))
RandomForest = make_pipeline(preprocessor, RandomForestClassifier(random_state=0))
DecisionTree = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=0))
AdaBoost = make_pipeline(preprocessor, AdaBoostClassifier(random_state=0))
SVM = make_pipeline(preprocessor,StandardScaler(), SVC(random_state=0))
KNN = make_pipeline(preprocessor, StandardScaler(),  KNeighborsClassifier())

DictionaireModel = {
    'RandomForest':RandomForest, 'DecisionTree':DecisionTree, 'AdaBoost':AdaBoost, 'SVM':SVM,'KNN':KNN
}
ListeModel = [RandomForest,DecisionTree,AdaBoost,SVM,KNN]

for nom,model in DictionaireModel.items():
    print(nom)
    evaluation(model)

In [ ]:
RandomForest = make_pipeline(preprocessor, RandomForestClassifier(random_state=0, n_estimators =110, criterion="gini",max_leaf_nodes =40))
evaluation(RandomForest)

In [ ]:
AdaBoost = make_pipeline(preprocessor, AdaBoostClassifier(random_state=0,n_estimators =110,algorithm="SAMME.R"))
evaluation(AdaBoost)

# Comparaison des deux modèle :
# La courbe d'apprentissage de l'adaboost est meilleur que celle du randomForest
# Les résultat du randomForest sont bien supérieur à ceux de l'adaboost. Il convient donc de choisir le randomForest

# Résultat : score = 81%, précision mort = 73%, précision survivant = 93%, recall mort = 94%, recall survivant = 70% ==> score f1 mort = 82% et score f1 survivant =80%